In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import re

from interpolate_to_common_grid_working import interpolate_to_common_grid
from find_error_between_vectors import error_of_vectors
from write_vtk_files import write_error_vector_to_vtk, write_vector_to_vtk

data_ind = "C:/Users/spyware/Documents/GitHub/MCG-4139/Final_project/Submission/Python Code/time_study/data/"
data_folders = os.listdir(data_ind)

In [2]:
##############################################################################
# Import min/max data
min_max_files = os.listdir(data_ind+data_folders[1]) 
df_min_max = pd.DataFrame(columns = ['CFL', 'Height_min', 'Height_max', 'X_velocity_min', 
                                     'X_velocity_max','Y_velocity_min', 'Y_velocity_max'])
for i in range(len(min_max_files)):
    location = data_ind + data_folders[1] + "/" + min_max_files[i]
    tmp_df = pd.read_fwf(location, 
                         widths=[30,30,30,30,30,30],
                         index_col=None)  
    tmp_df['CFL'] = min_max_files[i].rstrip(".dat").replace('vector_min_max_CFL_', '')
    df_min_max = pd.concat([df_min_max, tmp_df])
df_min_max = df_min_max.sort_values(by='CFL', ascending=False)
df_min_max = df_min_max.reset_index(drop=True)   

In [3]:
##############################################################################
# Import vector data
vector_files = os.listdir(data_ind+data_folders[3]) 
df_vector = pd.DataFrame(columns = ['CFL', 'X', 'Y', 'Height', 
                                          'X_velocity','Y_velocity'])
for i in range(len(min_max_files)):
    location = data_ind + data_folders[3] + "/" + vector_files[i]
    tmp_df = pd.read_fwf(location, 
                         widths=[30,30,30,30,30],
                         index_col=None)  
    tmp_df['CFL'] = vector_files[i].rstrip(".dat").replace('vector_CFL_', '')
    df_vector = pd.concat([df_vector, tmp_df])
df_vector = df_vector.sort_values(by='CFL', ascending=False)
df_vector = df_vector.reset_index(drop=True)   

In [4]:
#tmp_df = df_vector[df_vector['CharLen'].isin(['0.00075'])]
n = 1250
common_grid_shape = (n, n)
##############################################################################
# Interpolate solution vectors
interpolated_vector_df = interpolate_to_common_grid(df_vector, common_grid_shape)

Interpolating: 0.75
Interpolating: 0.5
Interpolating: 0.25


In [5]:
##############################################################################
# Calculate errors
error_vector_df = error_of_vectors(interpolated_vector_df, df_vector)

Calculating error: 0.75 vs. 0.5
Calculating error: 0.5 vs. 0.25


In [6]:
##############################################################################
# Find sum of absolute value of the error vectors
error_sum_dict = {}
for comparison in error_vector_df['CFL'].unique():
    df_comparison = error_vector_df[error_vector_df['CFL'] == comparison]
    height_error_sum = df_comparison['Height_error'].abs().sum()
    x_velocity_error_sum = df_comparison['X_velocity_error'].abs().sum()
    y_velocity_error_sum = df_comparison['Y_velocity_error'].abs().sum()
    error_sum_dict[comparison] = {'Height_error_sum': height_error_sum,
                                  'X_velocity_error_sum': x_velocity_error_sum,
                                  'Y_velocity_error_sum': y_velocity_error_sum}
error_sum_df = pd.DataFrame.from_dict(error_sum_dict, orient='index')

In [ ]:
error_sum_df

In [9]:
##############################################################################
# Export interpolated vectors to VTK
char_len_output = interpolated_vector_df['CFL'].unique()
for output in char_len_output:
    df_interp_output = interpolated_vector_df[interpolated_vector_df['CFL'] == output]
    filename = f"VTKs/Interpolated/{output}.vtk"
    write_vector_to_vtk(df_interp_output, filename)

Writing: VTKs/Interpolated/0.75.vtk
Writing: VTKs/Interpolated/0.5.vtk
Writing: VTKs/Interpolated/0.25.vtk


In [10]:
##############################################################################
# Export error vector to VTK
char_len_comparisons = error_vector_df['CFL'].unique()
for comparison in char_len_comparisons:
    df_comparison = error_vector_df[error_vector_df['CFL'] == comparison]
    filename = f"VTKs/Error/{comparison}.vtk"
    write_error_vector_to_vtk(df_comparison, filename)

Writing: VTKs/Error/0.75 vs. 0.5.vtk
Writing: VTKs/Error/0.5 vs. 0.25.vtk


In [ ]:
df_min_max_CharLens = df_min_max['CFL'].copy()
df_min_max_diff = df_min_max.drop(['CFL'], axis=1)
df_min_max_diff = df_min_max_diff.diff()
df_min_max_diff['CFL'] = df_min_max_CharLens

In [ ]:
df_min_max['Y_velocity_max']

In [ ]:
df_min_max_diff

In [ ]:
df_min_max['Y_velocity_max']

In [ ]:
df_min_max